# Case study 3: The Escherichia coli K-12 transcriptional motif

This is case study 3 in Figure 7 in this paper: *Eliater: an open source software for causal query estimation from observational measurements of biomolecular networks. The figure below shows the transcriptional regulatory network motif of E. coli from the EcoCyc database (Keseler et al., 2021). It comprises 44 nodes, of which 7 are latent and 45 are
observed, connected by a total of 147 directed edges.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from eliater import version_df
from eliater.data import load_ecoli_obs
from eliater.discover_latent_nodes import find_nuisance_variables, remove_nuisance_variables
from eliater.examples import ecoli_transcription_example
from eliater.network_validation import print_graph_falsifications
from eliater.regression import estimate_query, fit_regression, summary_statistics
from y0.algorithm.identify import Identification, identify_outcomes
from y0.dsl import P, Variable

version_df()

,key,value
0,eliater,0.0.1-dev-554fd195
1,y0,0.2.8-dev-05cee105
2,Run at,2024-01-19 09:29:23


In [2]:
treatment = Variable("fur")
outcome = Variable("dpiA")

In [3]:
graph = ecoli_transcription_example.graph

The observational data are 260 RNA-seq normalized expression profiles of E. coli K-12 MG1655 and BW25113 strains, spanning 154 distinct experimental conditions. These data were retrieved from the PRECISE database (Sastry et al., 2019).

In [4]:
data = load_ecoli_obs()
data.head()

,Unnamed: 0,cra,aceE,phoB,cyoA,amtB,appY,citC,dpiB,dpiA,...,mdh,fis,crp,rpoH,gadX,cspA,oxyR,soxS,dcuR,arcA
0,1,7.697991,10.651307,5.599989,11.254009,5.128441,5.919468,0.000000,4.323397,4.761679,...,11.745289,8.936196,9.666416,8.725102,8.614542,7.743336,8.209081,6.230069,5.752131,9.547993
1,2,7.581105,10.761484,5.746619,11.431429,5.614179,5.628823,0.000000,4.706430,4.930888,...,11.843169,8.601549,9.621358,8.769579,8.456629,7.474558,8.417561,6.104497,5.667736,9.560289
2,3,6.395808,12.109947,6.009650,9.937863,3.866721,5.789442,0.000000,4.776288,5.291245,...,11.166194,7.013285,9.761605,9.649581,8.391347,3.629759,7.841489,8.009047,5.725878,10.737478
3,4,6.366865,11.992996,6.131477,9.838573,3.666917,5.496674,0.377771,4.659630,5.366084,...,11.030617,7.078497,9.706964,9.746059,8.180204,4.538549,7.754378,8.172509,5.470773,10.788613
4,5,7.315204,10.569824,5.158324,11.272986,6.146377,5.275278,0.190837,4.332369,4.786446,...,11.423517,9.878379,9.692445,8.226875,8.142990,7.147757,7.979421,6.161353,5.738662,9.470079


## Step 1: Verify correctness of the network structure

The total number of d-separations implied by this network is large. It takes several hours to find all of them and test them against data. On the other hand, High-dimensional conditional independence tests can be very unreliable. Hence, it is best to control number of variables to condition upon. Here we selected 3. However, the default in this function is 5. Even with max_given equal to 3, it takes around an hour to produce the results.

In [ ]:
print_graph_falsifications(
    graph, data, max_given=3, method="pearson", verbose=True, significance_level=0.01
)

## Step 2: Check query identifiability

In [ ]:
identify_outcomes(graph=graph, treatments=treatment, outcomes=outcome)

The query is identifiable. Hence, we can proceed to the next step.

## Step 3: Find nuisance variables and mark them as latent

This function finds the nuisance variables for the input graph.

In [ ]:
nuisance_variables = find_nuisance_variables(graph, treatments=treatment, outcomes=outcome)
print(f"{len(nuisance_variables)} nuisance variables were found")
sorted(nuisance_variables)

## Step 4: Simplify the network

The following function find the nuisance variable (step 3), marks them as latent and then applies Evan's simplification rules to remove the nuisance variables.

In [ ]:
new_graph = remove_nuisance_variables(graph, treatments=treatment, outcomes=outcome)
new_graph.draw()

## Step 5: Estimate the query

In [ ]:
fit_regression(new_graph, data, treatments=treatment, outcome=outcome)

In [ ]:
estimate_query(
    new_graph,
    data,
    treatments=treatment,
    outcome=outcome,
    interventions={treatment: 0},
    query_type="expected_value",
)

The estimated value for the query $\mathbb{E}[\text{dpiA} \mid \text{fur} = 0]$ is 4.41. This shows that perturbation of gene fur has an increasing effect on gene dpiA.

In [ ]:
summary_statistics(
    new_graph,
    data,
    treatments=treatment,
    outcome=outcome,
    interventions={treatment: 0},
)

The summary statistics provides more details regarding the query in the form of $P(\text{dpiA} \mid \text{fur}=0)$. This result can be used to generate box plots that shows the estimated distribution of dpiA given a perturbation on fur.

## Evaluation criterion
The PRECISE database that the observational data was retrieved from included an interventional experiment with the value of fur set to zero, and three biological replicates. Hence, the ground truth for the query $\mathbb{E}[\text{dpiA} \mid \text{fur} = 0]$ was the mean expression of dpiA over the three replicates, resulted in 4.9.

The estimated value for the query (4.41) is close in sign and magnitude to the
value of 4.9 obtained from the experimental interventional data.

In [ ]:
x = estimate_query(
    new_graph,
    data,
    treatments=treatment,
    outcome=outcome,
    interventions={treatment: 0},
    query_type="probability",
)
ax = sns.boxplot(x=x)
ax.axvline(x=4.9, label="True expected value")
ax.legend()
ax.set(xlabel="Probability", title="Estimated outcome probability distribution")
plt.show()